# OPTICAL FLOW

In [15]:
import numpy as np
import cv2

In [30]:
# Dictionary to track corner parameters:
corner_track_params = dict(maxCorners = 10,qualityLevel=0.3,minDistance=7,blockSize = 7)

In [31]:
# Parameters for Lucas-Kanave

# Small window - more sensitive to noise, and miss larger motions.
# Larger window size - can detect larger motions, 
# but not sensitive to smaller motions.
# maxLevel - Allow us to see optical flow at various levels.
# criteria - EXCHANGE SPEED VS. ACCURACY

lk_params = dict(winSize=(200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0,0.3)) 



# CODING FROM UDEMY: 
# does not work...

In [34]:


cap = cv2.VideoCapture(0)

# Read first frame as previous frame
ret, prev_frame = cap.read() 

# Grab grayscale version:
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# PTS TO TRACK
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

# Creates a mask to draw on later:
mask = np.zeros_like(prev_frame)

while True:
    
    # NEXT FRAME:
    ret, frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame,
                        cv2.COLOR_BGR2GRAY)
    
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, # prev
                                                   frame_gray, # current
                                                    prevPts,
                                                   None, # Not define next points 
                                                    **lk_params) # Dictionary
    # Status vector == 1 if:
    # - Flow of corresponding features are found (after matching from index)
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    ########################################################
    
    # SUMMARY:
    # - Going through good_new points and good_prev points.
    # - Use enumerate to inerate through those points.
    
    
    
    
    
    for i, (new,prev) in enumerate(zip(good_new, good_prev)):
        
        x_new, y_new = new.ravel() # numpy method to flatten out an array
        x_prev, y_prev = prev.ravel()
        
        # Draw line on mask from (x_new, y_new) to (x_prev, y_prev):
        mask = cv2.line(mask, (x_new, y_new),
                       (x_prev, y_prev),
                       (0,255,0), 3)
        
        # Draw dot/circle on new points:
        frame = cv2.circle(frame, (x_new, y_new), 
                      8, (0,0,255), -1)
        # Radius = 8, Red, Filled in.
    
    img = cv2.add(frame, mask) # add on frame and mask
    cv2.imshow('tracking', img) # Show frame with points and added mask.
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()
cap.release()


SyntaxError: positional argument follows keyword argument (<ipython-input-34-2f0d0b99dc41>, line 10)

# THIS ONE ACTUALLY WORKS:

In [36]:
# LINK: https://github.com/vishnurapps/OpenCV/blob/master/python-for-computer-vision-with-opencv-and-deep-learning/06_ObjectTracking/01_LucasKanadeOpticalFlow.py

import numpy as np
import cv2 

# Parameters for ShiTomasi corner detection (good features to track paper)
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7 )

# Parameters for Lucas Kanade Optimal Flow
lk_params = dict( winSize  = (200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT, 10,0.03))

# Capture the video
cap = cv2.VideoCapture(0)

# Grab the very first frame of the stream
ret, prev_frame = cap.read()

# Grab a grayscale image (We will refer to this as the previous frame)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Grabbing the corners
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# Create a matching mask of the previous frame for drawing on later
mask = np.zeros_like(prev_frame)


while True:
    
    # Grab current frame
    ret,frame = cap.read()
    
    # Grab gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Optical Flow on the Gray Scale Frame
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    # Use ravel to get points to draw lines and circles
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()